<a href="https://colab.research.google.com/github/monicagokul1812/Mini-project-1/blob/main/earthquake_project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Project title : Earthquake data extraction**

In [17]:
# Project title : Earthquake data extraction
# Global Seismic Trends: Data-Driven Earthquake Insights.
#Goal: Identify high-risk zones & trends using real earthquake data (2020–2025)
#streamlit (target)



# **FETCH THE DATA FROM USGS API**

In [18]:
url = "https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2020-01-01&endtime=2025-01-01 &minmagnitude=3"

# **Added** **library**



1.Preprocessing and cleaning the raw data.
2.feature engineering .

1.handling missing value.
2.scaling & normalize the data.



In [19]:
import requests
import pandas as pd
from datetime import datetime
import time
import os
import re

BASE_PATH = "C:/New folder/data"
os.makedirs(BASE_PATH, exist_ok=True)

RAW_CSV_PATH = f"{BASE_PATH}/raw_earthquakes.csv"
CLEANED_CSV_PATH = f"{BASE_PATH}/cleaned_earthquakes.csv"

url = "https://earthquake.usgs.gov/fdsnws/event/1/query"

all_records = []
start_year = 2020
end_year = 2025

for year in range(start_year, end_year + 1):
    for month in range(1, 13):
        start_date = f"{year}-{month:02d}-01"

        if month == 12:
            end_date = f"{year+1}-01-01"
        else:
            end_date = f"{year}-{month+1:02d}-01"

        params = {
            "format": "geojson",
            "starttime": start_date,
            "endtime": end_date,
            "minmagnitude": 3
        }

        response = requests.get(url, params=params)

        if response.status_code != 200:
            print("Request failed:", response.text[:200])
        else:
            data = response.json()
            if "features" in data:
                for feature in data["features"]:
                    prop = feature["properties"]
                    geom = feature["geometry"]

                    all_records.append({
                        "id": feature["id"],
                        "time": prop["time"],
                        "updated": prop["updated"],
                        "mag": prop["mag"],
                        "magType": prop["magType"],
                        "place": prop["place"],
                        "status": prop["status"],
                        "tsunami": prop["tsunami"],
                        "sig": prop["sig"],
                        "net": prop["net"],
                        "nst": prop.get("nst"),
                        "dmin": prop.get("dmin"),
                        "rms": prop.get("rms"),
                        "gap": prop.get("gap"),
                        "magError": prop.get("magError"),
                        "depthError": prop.get("depthError"),
                        "magNst": prop.get("magNst"),
                        "locationSource": prop.get("locationSource"),
                        "magSource": prop.get("magSource"),
                        "types": prop["types"],
                        "ids": prop["ids"],
                        "sources": prop["sources"],
                        "type": prop["type"],
                        "longitude": geom["coordinates"][0],
                        "latitude": geom["coordinates"][1],
                        "depth_km": geom["coordinates"][2]
                    })

df = pd.DataFrame(all_records)
df.to_csv(RAW_CSV_PATH, index=False)
print("Raw data saved successfully")

df = pd.read_csv(RAW_CSV_PATH)

df["time"] = pd.to_datetime(df["time"], unit="ms")
df["updated"] = pd.to_datetime(df["updated"], unit="ms")

def extract_country(place):
    if pd.isna(place):
        return "Unknown"
    match = re.search(r",\s*(.*)$", place)
    return match.group(1) if match else "Unknown"

df["country"] = df["place"].apply(extract_country)


numeric_cols = [
    "mag", "depth_km", "nst", "dmin", "rms",
    "gap", "magError", "depthError", "magNst", "sig"
]

for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce").fillna(0)

df["year"] = df["time"].dt.year
df["month"] = df["time"].dt.month
df["day"] = df["time"].dt.day
df["day_of_week"] = df["time"].dt.day_name()

df["depth_category"] = df["depth_km"].apply(
    lambda x: "Shallow" if x < 50 else "Deep"
)

df["strength"] = df["mag"].apply(
    lambda x: "Strong" if x >= 7 else "Moderate"
)

df.to_csv(CLEANED_CSV_PATH, index=False)

print("Data cleaned and saved successfully")
print(f"Cleaned file location: {CLEANED_CSV_PATH}")




Raw data saved successfully
Data cleaned and saved successfully
Cleaned file location: C:/New folder/data/cleaned_earthquakes.csv


# **DATA INSERTED INTO MYSQL**

In [ ]:
import pandas as pd
import mysql.connector
import math

# Load cleaned CSV
df = pd.read_csv("C:/New folder/data/cleaned_earthquakes.csv")

# Connect to MySQL
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="root",
    database="earthquake_db"
)

cursor = conn.cursor()

insert_query = """
INSERT IGNORE INTO earthquakes (
    id, `time`, updated, latitude, longitude, depth_km,
    mag, magType, place, country, status, tsunami, sig, net,
    nst, dmin, rms, gap, magError, depthError, magNst,
    locationSource, magSource, types, ids, sources, `type`
)
VALUES (%s, %s, %s, %s, %s, %s,
        %s, %s, %s, %s, %s, %s, %s, %s,
        %s, %s, %s, %s, %s, %s, %s,
        %s, %s, %s, %s, %s, %s)
"""

count = 0

for _, row in df.iterrows():

    values = tuple(
        None if (isinstance(v, float) and math.isnan(v)) else v
        for v in (
            row["id"], row["time"], row["updated"],
            row["latitude"], row["longitude"], row["depth_km"],
            row["mag"], row["magType"], row["place"], row["country"],
            row["status"], row["tsunami"], row["sig"], row["net"],
            row["nst"], row["dmin"], row["rms"], row["gap"],
            row["magError"], row["depthError"], row["magNst"],
            row["locationSource"], row["magSource"],
            row["types"], row["ids"], row["sources"], row["type"]
        )
    )

    cursor.execute(insert_query, values)
    count += 1

    if count % 1000 == 0:
        conn.commit()
        print(f"{count} rows inserted...")
conn.commit()
cursor.close()
conn.close()

print("All data inserted successfully!")
